In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from torchvision import datasets,transforms
from sklearn.model_selection import train_test_split

import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

In [2]:
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ]))

In [3]:
train_loader = DataLoader(train_data,  batch_size=100, shuffle=True)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(256, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.reshape(-1, 256)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x)

model = Net()
model = model.cuda()
torch.backends.cudnn.benchmark=True

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [6]:
for epoch in range(10):
    total_loss = 0
    for data, target in train_loader:
        #CNN filter에 들어가는값 [batch, filters, heigth, width]
        train_x, train_y = Variable(data.reshape(-1, 1, 28, 28).float()).cuda(), Variable(target.long()).cuda()
        optimizer.zero_grad()
        output = model(train_x)
        loss = criterion(output, train_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    if(epoch + 1) %2 == 0:
        print(epoch+1, total_loss)

C:\Users\my\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


2 0.14100906252861023
4 0.07714777439832687
6 0.11994470655918121
8 0.08279245346784592
10 0.1684304177761078


In [12]:
test_x, test_y = Variable(test_data.test_data.float()).cuda(), Variable(test_data.test_labels.long()).cuda()
test_x = test_x.reshape(-1, 1,28,28)

In [13]:
result = torch.max(model(test_x).data, 1)[1]
accuracy = sum(test_y.cpu().numpy() == result.cpu().numpy()) / len(test_y.cpu().numpy())
accuracy

C:\Users\my\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


0.9801